Imports.

In [1]:
import numpy as np

import pandas as pd

import gensim
from gensim.models import Word2Vec as w2v

from ast import literal_eval

import time

Load DataFrame.

In [2]:
chaps_df = pd.read_csv('../data/animorphs_chaps.csv')

for col in ['clean','clean_no_stops']:
    chaps_df[col] = chaps_df[col].map(literal_eval) # as lists
    
chaps_df.head()

,book,chapter,text,authenticity,clean,clean_no_stops,vec_clean
0,1,1,"My name is Jake. That's my first name, obvious...",1,"[name, thats, first, name, obviously, cant, te...","[my, name, is, jake, thats, my, first, name, o...",[-0.06084491 -0.07821839 0.1802005 -0.032568...
1,1,2,"""A flying saucer?"" Marco said. He did laugh. T...",1,"[flying, saucer, laugh, looked, could, feel, h...","[a, flying, saucer, marco, said, he, did, laug...",[-0.07276286 -0.0811749 0.17935286 -0.038883...
2,1,3,<They have come to destroy you.>\nIt was stran...,1,"[come, destroy, strange, way, knew, telling, t...","[they, have, come, to, destroy, you, it, was, ...",[-1.03591643e-01 -9.50562879e-02 1.48249313e-...
3,1,4,<Yeerks!>\nThe twin red lights slowed. They tu...,1,"[yeerks, twin, red, lights, slowed, turned, ci...","[yeerks, the, twin, red, lights, slowed, they,...",[-1.19802251e-01 -1.08502194e-01 1.65713996e-...
4,1,5,"The Hork-Bajir pointed his gun, or whatever it...",1,"[hork, bajir, pointed, gun, whatever, around, ...","[the, hork, bajir, pointed, his, gun, or, what...",[-1.01123430e-01 -1.02614835e-01 1.57715589e-...


A function to create or load a word vector model.

In [3]:
vsize=300 # size of all w2v vectors

def create_vectors(vector_file, col='clean', 
                   chaps_df=chaps_df, reset=False):
    """
    A function to set up word vectorization for a given corpus,
    or load it from a given file.
    """
    try: # load word vectors
        
        assert not reset
        
        w2vmodel = w2v.load(vector_file)

    except: # create word vectors

        t0 = time.time() # this might be a while

        # make corpus
        corpus = list(chaps_df[col])

        w2vmodel = w2v(corpus,
                       size=vsize, # somewhat arbitrary, plecháč used 100 d
                       # could gridsearch over most of these params, but
                       # would take forever. instead, make informed decision
                       window=5, 
                       min_count=2,
                       sg=1, # skipgram
                       workers=3)
        
        w2vmodel.save(vector_file)   
        
        print('It took',time.time()-t0,'seconds to generate this model.')
    
    return w2vmodel

In [4]:
vector_file = '../data/animorphs.vector'

w2vmodel = create_vectors(vector_file)

In [5]:
w2vmodel.wv.most_similar('planet')

[('invaded', 0.8538174629211426),
 ('invading', 0.843326210975647),
 ('leera', 0.8390116691589355),
 ('universe', 0.8216259479522705),
 ('planets', 0.8168292045593262),
 ('galaxy', 0.8142693042755127),
 ('earth', 0.812616765499115),
 ('conquest', 0.8082102537155151),
 ('resistance', 0.8065967559814453),
 ('throughout', 0.805733323097229)]

In [6]:
np.mean([w2vmodel.wv.word_vec('planet'), w2vmodel.wv.word_vec('earth')], axis=0).shape

(300,)

This next cell was inextricably influenced by the work of Boom Devahastin Na Ayudhya, in the function `vectorize_corpus` available [here](https://github.com/boom-deva/FEMA-Power-Outage-Hotspot-Detection/blob/master/code/4_Preprocessing-and-NLP-Modeling.ipynb).

Define a function to get the average vector of a list of words, ignoring unfamiliar words.

In [7]:
# influenced by boom's vectorize_corpus
def avg_vec_words(words, w2vmodel=w2vmodel):
    """
    A function to get the average vector of a list of words.
    Ignores unfamiliar words.
    """
    vecs = [w2vmodel.wv.word_vec(w)    # vectorize word w
            for w in words             # for all words
            if w in w2vmodel.wv.vocab] # that are in the vocab
    
# below code would treat unfamiliar words as 0 vectors instead of ignoring
#     vecs = [w2vmodel.wv.word_vec(w)    # vectorize word w
#             if w in w2vmodel.wv.vocab  # if w can be vectorized
#             else np.zeros(vsize)       # otherwise give it a 0 vector
#             for w in words]            # for all words
    
    return np.mean(vecs, axis=0) # return avg of vecs

Vectorize each chapter.

In [8]:
chaps_df['vec_clean'] = chaps_df['clean'].map(avg_vec_words)

chaps_df.head()

,book,chapter,text,authenticity,clean,clean_no_stops,vec_clean
0,1,1,"My name is Jake. That's my first name, obvious...",1,"[name, thats, first, name, obviously, cant, te...","[my, name, is, jake, thats, my, first, name, o...","[-0.06084491, -0.07821839, 0.1802005, -0.03256..."
1,1,2,"""A flying saucer?"" Marco said. He did laugh. T...",1,"[flying, saucer, laugh, looked, could, feel, h...","[a, flying, saucer, marco, said, he, did, laug...","[-0.07276286, -0.081174895, 0.17935286, -0.038..."
2,1,3,<They have come to destroy you.>\nIt was stran...,1,"[come, destroy, strange, way, knew, telling, t...","[they, have, come, to, destroy, you, it, was, ...","[-0.10359164, -0.09505629, 0.14824931, -0.0206..."
3,1,4,<Yeerks!>\nThe twin red lights slowed. They tu...,1,"[yeerks, twin, red, lights, slowed, turned, ci...","[yeerks, the, twin, red, lights, slowed, they,...","[-0.11980225, -0.108502194, 0.165714, -0.03714..."
4,1,5,"The Hork-Bajir pointed his gun, or whatever it...",1,"[hork, bajir, pointed, gun, whatever, around, ...","[the, hork, bajir, pointed, his, gun, or, what...","[-0.10112343, -0.102614835, 0.15771559, -0.031..."


Create a DataFrame of two potential vectorizations of books.

`book_vec` is the simple averge vectorization of every word in the book.
`avg_chap_vec` is the average of the vectorization of each chapter (which is itself the average vectorization of every word in the chapter), which would be inappropriate for normal use but might be relevant in chapter-based classification methods.

In [ ]:
books_gb = chaps_df.groupby(by='book')

books_df = pd.DataFrame(data={
    'book':books_gb['book'].mean(), # book number
    'book_vec':[avg_vec_words(
        [w 
         for c in chaps_df.loc[chaps_df['book']==b, 'clean'] 
         for w in c]) for b in chaps_df['book'].unique()], # book vector
    'avg_chap_vec':books_gb['vec_clean'].apply(
        lambda v: np.mean(v, axis=0)), # avg vector of chapters in book
    'authenticity':books_gb['authenticity'].mean() # authenticity
})

books_df.head()

Save updated DataFrames.

In [ ]:
chaps_df.to_csv('../data/animorphs_chaps.csv', index=False)
books_df.to_csv('../data/animorphs_books.csv', index=False)